In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import threading 
import datetime
import importlib

In [2]:
from dnnTorerosModel import *

2022-06-03 11:44:42.911961: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-03 11:44:42.911981: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/wesselweernink/VENVS/pic_b/lib/python3.10/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [3]:
#importlib.reload(tm)

In [4]:
nasdaq_financials = pd.read_csv("comp_all_fin_nasdaq.csv")

In [5]:
nasdaq_financials

,Symbol,Date,Research Development,Effect Of Accounting Charges,Income Before Tax,Minority Interest,Net Income,Selling General Administrative,Gross Profit,Ebit,...,Discontinued Operations_lag_2,Net Income From Continuing Ops_lag_1,Net Income From Continuing Ops_lag_2,Net Income Applicable To Common Shares_lag_1,Net Income Applicable To Common Shares_lag_2,Close_lag_1,Close_lag_2,Volume_lag_1,Volume_lag_2,FutureDirection
0,AAME,2020-12-31,0.000000e+00,0,1.550300e+07,0.000000e+00,1.216900e+07,0.000000e+00,2.866100e+07,1.711300e+07,...,0.0,-3.860000e+05,-7.040000e+05,-7.850000e+05,-1.103000e+06,1.948998,2.365385,3400.0,5900.0,1.195531
1,AAOI,2020-12-31,4.339300e+07,0,-5.122400e+07,0.000000e+00,-5.845200e+07,5.599000e+07,5.054100e+07,-4.884200e+07,...,0.0,-6.604900e+07,-2.146000e+06,-6.604900e+07,-2.146000e+06,11.880000,15.430000,559800.0,774100.0,0.603995
2,AAON,2020-12-31,0.000000e+00,0,1.019750e+08,0.000000e+00,7.900900e+07,5.637700e+07,1.558490e+08,9.747300e+07,...,0.0,5.371100e+07,4.232900e+07,5.371100e+07,4.232900e+07,48.837917,34.423103,178900.0,154800.0,1.198429
3,AAPL,2020-09-26,1.875200e+10,0,6.709100e+10,0.000000e+00,5.741100e+10,1.991600e+10,1.049560e+11,6.628800e+10,...,0.0,5.525600e+10,5.953100e+10,5.525600e+10,5.953100e+10,53.674576,54.525856,101408000.0,91717600.0,1.316931
4,AATC,2020-12-31,3.336000e+06,0,6.010000e+05,0.000000e+00,1.063000e+06,6.444000e+06,1.038800e+07,6.080000e+05,...,0.0,6.995000e+06,1.862000e+06,6.995000e+06,1.862000e+06,4.252031,4.214569,2400.0,1500.0,1.472303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2359,YI,2019-12-31,6.190200e+07,0,-5.018270e+08,-2.733000e+06,-4.996060e+08,4.640630e+08,3.618700e+07,-4.935130e+08,...,0.0,-3.820410e+08,-2.493270e+08,-3.800910e+08,-2.485800e+08,6.140000,13.800000,4700.0,1302200.0,1.032689
2360,YMAB,2020-12-31,9.258300e+07,0,-1.193370e+08,0.000000e+00,-1.193370e+08,4.589900e+07,1.854700e+07,-1.199350e+08,...,0.0,-8.102800e+07,-4.327400e+07,-8.102800e+07,-4.327400e+07,31.250000,20.340000,133600.0,160900.0,0.327409
2361,YNDX,2019-12-31,2.920900e+10,0,2.285500e+10,3.450700e+10,1.282600e+10,5.015500e+10,1.196030e+11,2.546200e+10,...,0.0,4.425800e+10,9.081000e+09,4.598400e+10,9.201000e+09,27.350000,33.639999,931200.0,2087000.0,1.599908
2362,YORW,2019-12-31,0.000000e+00,0,1.664200e+07,0.000000e+00,1.440200e+07,9.968000e+06,4.119100e+07,2.233500e+07,...,0.0,1.337600e+07,1.297400e+07,1.337600e+07,1.297400e+07,30.746683,31.291149,17900.0,39100.0,1.026672


In [6]:
nasdaq_financials.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2364 entries, 0 to 2363
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Symbol                                        2364 non-null   object 
 1   Date                                          2364 non-null   object 
 2   Research Development                          2364 non-null   float64
 3   Effect Of Accounting Charges                  2364 non-null   int64  
 4   Income Before Tax                             2364 non-null   float64
 5   Minority Interest                             2364 non-null   float64
 6   Net Income                                    2364 non-null   float64
 7   Selling General Administrative                2364 non-null   float64
 8   Gross Profit                                  2364 non-null   float64
 9   Ebit                                          2364 non-null   f

In [7]:
finan_stcks = nasdaq_financials.copy() 

In [8]:
finan_stcks["PredictionGroup"] = np.where( finan_stcks["FutureDirection"]>= 1.15, 4, None )
finan_stcks["PredictionGroup"] = np.where( finan_stcks["FutureDirection"]< 1.15, 3, finan_stcks["PredictionGroup"] )
finan_stcks["PredictionGroup"] = np.where( finan_stcks["FutureDirection"]< 1.04, 2, finan_stcks["PredictionGroup"])
finan_stcks["PredictionGroup"] = np.where( finan_stcks["FutureDirection"]< 0.96, 1, finan_stcks["PredictionGroup"])
finan_stcks["PredictionGroup"] = np.where( finan_stcks["FutureDirection"]< 0.85, 0, finan_stcks["PredictionGroup"])

In [9]:
split = int(len(nasdaq_financials) * 0.8)
split

1891

In [10]:
train = finan_stcks.iloc[:split].copy()

In [11]:
train

,Symbol,Date,Research Development,Effect Of Accounting Charges,Income Before Tax,Minority Interest,Net Income,Selling General Administrative,Gross Profit,Ebit,...,Net Income From Continuing Ops_lag_1,Net Income From Continuing Ops_lag_2,Net Income Applicable To Common Shares_lag_1,Net Income Applicable To Common Shares_lag_2,Close_lag_1,Close_lag_2,Volume_lag_1,Volume_lag_2,FutureDirection,PredictionGroup
0,AAME,2020-12-31,0.000000e+00,0,1.550300e+07,0.0,1.216900e+07,0.000000e+00,2.866100e+07,1.711300e+07,...,-3.860000e+05,-7.040000e+05,-7.850000e+05,-1.103000e+06,1.948998,2.365385,3400.0,5900.0,1.195531,4
1,AAOI,2020-12-31,4.339300e+07,0,-5.122400e+07,0.0,-5.845200e+07,5.599000e+07,5.054100e+07,-4.884200e+07,...,-6.604900e+07,-2.146000e+06,-6.604900e+07,-2.146000e+06,11.880000,15.430000,559800.0,774100.0,0.603995,0
2,AAON,2020-12-31,0.000000e+00,0,1.019750e+08,0.0,7.900900e+07,5.637700e+07,1.558490e+08,9.747300e+07,...,5.371100e+07,4.232900e+07,5.371100e+07,4.232900e+07,48.837917,34.423103,178900.0,154800.0,1.198429,4
3,AAPL,2020-09-26,1.875200e+10,0,6.709100e+10,0.0,5.741100e+10,1.991600e+10,1.049560e+11,6.628800e+10,...,5.525600e+10,5.953100e+10,5.525600e+10,5.953100e+10,53.674576,54.525856,101408000.0,91717600.0,1.316931,4
4,AATC,2020-12-31,3.336000e+06,0,6.010000e+05,0.0,1.063000e+06,6.444000e+06,1.038800e+07,6.080000e+05,...,6.995000e+06,1.862000e+06,6.995000e+06,1.862000e+06,4.252031,4.214569,2400.0,1500.0,1.472303,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1886,SCHL,2020-05-31,0.000000e+00,0,-8.970000e+07,1400000.0,-4.380000e+07,7.051000e+08,6.961000e+08,-7.050000e+07,...,1.560000e+07,-5.000000e+06,1.560000e+07,-5.000000e+06,31.710461,42.495613,490900.0,134300.0,1.198008,4
1887,SCHN,2020-08-31,0.000000e+00,0,-1.939000e+06,3729000.0,-4.145000e+06,1.878030e+08,2.086180e+08,2.081500e+07,...,5.857000e+07,1.594430e+08,5.634500e+07,1.564510e+08,20.740690,23.944056,114900.0,450000.0,2.452324,4
1888,SCKT,2019-12-31,3.893563e+06,0,5.057140e+05,0.0,2.865860e+05,5.600710e+06,1.010064e+07,6.063700e+05,...,-5.711410e+05,-1.430731e+06,-5.711410e+05,-1.430731e+06,1.500000,3.600000,25000.0,20400.0,1.478261,4
1889,SCOR,2020-12-31,3.609700e+07,0,-4.701600e+07,0.0,-4.791800e+07,1.321440e+08,1.808560e+08,-1.460400e+07,...,-3.389960e+08,-1.592680e+08,-3.389960e+08,-1.592680e+08,4.940000,14.430000,1109600.0,172400.0,1.341365,4


In [12]:
test = nasdaq_financials.iloc[split:].copy()

In [13]:
columns_use = list(nasdaq_financials.columns)

In [14]:
avoid = ["Symbol", "Date", "FutureDirection", "PredictionGroup"]


In [15]:
#using reversed here to avoid problem where python skips over certain elements after deleting items because index is
# changed. Using reversed makes sure that all elements are checked. By deleting from the end of the list the index 
# of other elements in the list don't get shifted to fill the gap that would appear in the index.
for col in reversed(columns_use):
    if col in avoid:
        columns_use.remove(col)

In [16]:
train = train[columns_use]

In [17]:
train

,Research Development,Effect Of Accounting Charges,Income Before Tax,Minority Interest,Net Income,Selling General Administrative,Gross Profit,Ebit,Operating Income,Other Operating Expenses,...,Discontinued Operations_lag_1,Discontinued Operations_lag_2,Net Income From Continuing Ops_lag_1,Net Income From Continuing Ops_lag_2,Net Income Applicable To Common Shares_lag_1,Net Income Applicable To Common Shares_lag_2,Close_lag_1,Close_lag_2,Volume_lag_1,Volume_lag_2
0,0.000000e+00,0,1.550300e+07,0.0,1.216900e+07,0.000000e+00,2.866100e+07,1.711300e+07,1.711300e+07,11548000.0,...,0.0,0.0,-3.860000e+05,-7.040000e+05,-7.850000e+05,-1.103000e+06,1.948998,2.365385,3400.0,5900.0
1,4.339300e+07,0,-5.122400e+07,0.0,-5.845200e+07,5.599000e+07,5.054100e+07,-4.884200e+07,-4.884200e+07,0.0,...,0.0,0.0,-6.604900e+07,-2.146000e+06,-6.604900e+07,-2.146000e+06,11.880000,15.430000,559800.0,774100.0
2,0.000000e+00,0,1.019750e+08,0.0,7.900900e+07,5.637700e+07,1.558490e+08,9.747300e+07,9.747300e+07,0.0,...,0.0,0.0,5.371100e+07,4.232900e+07,5.371100e+07,4.232900e+07,48.837917,34.423103,178900.0,154800.0
3,1.875200e+10,0,6.709100e+10,0.0,5.741100e+10,1.991600e+10,1.049560e+11,6.628800e+10,6.628800e+10,0.0,...,0.0,0.0,5.525600e+10,5.953100e+10,5.525600e+10,5.953100e+10,53.674576,54.525856,101408000.0,91717600.0
4,3.336000e+06,0,6.010000e+05,0.0,1.063000e+06,6.444000e+06,1.038800e+07,6.080000e+05,6.080000e+05,0.0,...,0.0,0.0,6.995000e+06,1.862000e+06,6.995000e+06,1.862000e+06,4.252031,4.214569,2400.0,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1886,0.000000e+00,0,-8.970000e+07,1400000.0,-4.380000e+07,7.051000e+08,6.961000e+08,-7.050000e+07,-7.050000e+07,0.0,...,0.0,0.0,1.560000e+07,-5.000000e+06,1.560000e+07,-5.000000e+06,31.710461,42.495613,490900.0,134300.0
1887,0.000000e+00,0,-1.939000e+06,3729000.0,-4.145000e+06,1.878030e+08,2.086180e+08,2.081500e+07,2.081500e+07,0.0,...,-248000.0,346000.0,5.857000e+07,1.594430e+08,5.634500e+07,1.564510e+08,20.740690,23.944056,114900.0,450000.0
1888,3.893563e+06,0,5.057140e+05,0.0,2.865860e+05,5.600710e+06,1.010064e+07,6.063700e+05,6.063700e+05,0.0,...,0.0,0.0,-5.711410e+05,-1.430731e+06,-5.711410e+05,-1.430731e+06,1.500000,3.600000,25000.0,20400.0
1889,3.609700e+07,0,-4.701600e+07,0.0,-4.791800e+07,1.321440e+08,1.808560e+08,-1.460400e+07,-1.460400e+07,0.0,...,0.0,0.0,-3.389960e+08,-1.592680e+08,-3.389960e+08,-1.592680e+08,4.940000,14.430000,1109600.0,172400.0


In [18]:
len(columns_use)

72

In [19]:
#columns_use.remove("FutureDirection")

In [20]:
#DONT use as it doesn't make sense to have normalization of this data as information about debt and relative cash level are lost.
#mu, std = train.mean(), train.std()

In [21]:
#std

In [22]:
#mu

In [23]:
#Don't use as explained above.
#train_stand = (train - mu)/ std

In [24]:
train_stand = train#.fillna(0)

In [25]:
pd.set_option("display.max_columns", None)

In [26]:
#train_stand

In [27]:
test = test[columns_use]

In [28]:
#mu2, std2 = test.mean(), test.std()

In [29]:
#test_stand = (test -mu2) / std2
test_stand = test#.fillna(0)

In [30]:
#test_stand

In [31]:
finan_stcks["PredictionGroup"][:split]

0       4
1       0
2       4
3       4
4       4
       ..
1886    4
1887    4
1888    4
1889    4
1890    1
Name: PredictionGroup, Length: 1891, dtype: object

In [32]:
finan_stcks["PredictionGroup"] = pd.to_numeric(finan_stcks["PredictionGroup"])

In [33]:
finan_stcks["PredictionGroup"]

0       4
1       0
2       4
3       4
4       4
       ..
2359    2
2360    0
2361    4
2362    2
2363    1
Name: PredictionGroup, Length: 2364, dtype: int64

In [34]:
finan_stcks[:split]

,Symbol,Date,Research Development,Effect Of Accounting Charges,Income Before Tax,Minority Interest,Net Income,Selling General Administrative,Gross Profit,Ebit,Operating Income,Other Operating Expenses,Interest Expense,Extraordinary Items,Non Recurring,Other Items,Income Tax Expense,Total Revenue,Total Operating Expenses,Cost Of Revenue,Total Other Income Expense Net,Discontinued Operations,Net Income From Continuing Ops,Net Income Applicable To Common Shares,Close,Volume,Research Development_lag_1,Research Development_lag_2,Effect Of Accounting Charges_lag_1,Effect Of Accounting Charges_lag_2,Income Before Tax_lag_1,Income Before Tax_lag_2,Minority Interest_lag_1,Minority Interest_lag_2,Net Income_lag_1,Net Income_lag_2,Selling General Administrative_lag_1,Selling General Administrative_lag_2,Gross Profit_lag_1,Gross Profit_lag_2,Ebit_lag_1,Ebit_lag_2,Operating Income_lag_1,Operating Income_lag_2,Other Operating Expenses_lag_1,Other Operating Expenses_lag_2,Interest Expense_lag_1,Interest Expense_lag_2,Extraordinary Items_lag_1,Extraordinary Items_lag_2,Non Recurring_lag_1,Non Recurring_lag_2,Other Items_lag_1,Other Items_lag_2,Income Tax Expense_lag_1,Income Tax Expense_lag_2,Total Revenue_lag_1,Total Revenue_lag_2,Total Operating Expenses_lag_1,Total Operating Expenses_lag_2,Cost Of Revenue_lag_1,Cost Of Revenue_lag_2,Total Other Income Expense Net_lag_1,Total Other Income Expense Net_lag_2,Discontinued Operations_lag_1,Discontinued Operations_lag_2,Net Income From Continuing Ops_lag_1,Net Income From Continuing Ops_lag_2,Net Income Applicable To Common Shares_lag_1,Net Income Applicable To Common Shares_lag_2,Close_lag_1,Close_lag_2,Volume_lag_1,Volume_lag_2,FutureDirection,PredictionGroup
0,AAME,2020-12-31,0.000000e+00,0,1.550300e+07,0.0,1.216900e+07,0.000000e+00,2.866100e+07,1.711300e+07,1.711300e+07,11548000.0,-1.610000e+06,0,0,0,3.334000e+06,1.953480e+08,1.782350e+08,1.666870e+08,-1610000.0,0.0,1.216900e+07,1.177000e+07,2.038039,7800.0,0.000000e+00,0.000000e+00,0.0,0.0,-4.070000e+05,-9.710000e+05,0.0,0.0,-3.860000e+05,-7.040000e+05,0.000000e+00,0.000000e+00,1.347700e+07,1.386100e+07,1.723000e+06,1.066000e+06,1.723000e+06,1.066000e+06,11754000.0,12795000.0,-2.130000e+06,-2.037000e+06,0.0,0.0,0.0,0.0,0.0,0.0,-2.100000e+04,-2.670000e+05,1.981790e+08,1.855530e+08,1.964560e+08,1.844870e+08,1.847020e+08,1.716920e+08,-2.130000e+06,-2.037000e+06,0.0,0.0,-3.860000e+05,-7.040000e+05,-7.850000e+05,-1.103000e+06,1.948998,2.365385,3400.0,5900.0,1.195531,4
1,AAOI,2020-12-31,4.339300e+07,0,-5.122400e+07,0.0,-5.845200e+07,5.599000e+07,5.054100e+07,-4.884200e+07,-4.884200e+07,0.0,-5.635000e+06,0,0,0,7.228000e+06,2.346230e+08,2.834650e+08,1.840820e+08,-2382000.0,0.0,-5.845200e+07,-5.845200e+07,8.510000,638400.0,4.339900e+07,4.990300e+07,0.0,0.0,-5.138700e+07,-9.778000e+06,0.0,0.0,-6.604900e+07,-2.146000e+06,5.154900e+07,4.863800e+07,4.620100e+07,8.777300e+07,-4.874700e+07,-1.076800e+07,-4.874700e+07,-1.076800e+07,0.0,0.0,-5.405000e+06,-1.106000e+06,0.0,0.0,0.0,0.0,0.0,0.0,1.466200e+07,-7.632000e+06,1.908720e+08,2.674650e+08,2.396190e+08,2.782330e+08,1.446710e+08,1.796920e+08,-2.640000e+06,9.900000e+05,0.0,0.0,-6.604900e+07,-2.146000e+06,-6.604900e+07,-2.146000e+06,11.880000,15.430000,559800.0,774100.0,0.603995,0
2,AAON,2020-12-31,0.000000e+00,0,1.019750e+08,0.0,7.900900e+07,5.637700e+07,1.558490e+08,9.747300e+07,9.747300e+07,0.0,-1.320000e+05,0,0,0,2.296600e+07,5.145510e+08,4.170780e+08,3.587020e+08,4502000.0,0.0,7.900900e+07,7.900900e+07,66.278435,172900.0,0.000000e+00,0.000000e+00,0.0,0.0,6.703100e+07,5.550000e+07,0.0,0.0,5.371100e+07,4.232900e+07,4.941600e+07,4.613700e+07,1.194250e+08,1.035330e+08,6.848500e+07,5.627200e+07,6.848500e+07,5.627200e+07,0.0,0.0,-1.320000e+05,-1.320000e+05,0.0,0.0,0.0,0.0,0.0,0.0,1.332000e+07,1.317100e+07,4.693330e+08,4.339470e+08,4.008480e+08,3.776750e+08,3.499080e+08,3.304140e+08,-1.454000e+06,-7.720000e+05,0.0,0.0,5.371100e+07,4.232900e+07,5.371100e+07,4.232900e+07,48.837917,34.423103,178900.0,154800.0,1

In [35]:
len(columns_use)

72

In [36]:
#############################################################
# creating model

set_seeds(101)
model = create_model(hl = 5, hu = 150, dropout = True, input_dim = len(columns_use), regularize = True)
model.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 400, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

2022-06-03 11:44:44.448782: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-03 11:44:44.449012: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-03 11:44:44.449071: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-06-03 11:44:44.449118: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-06-03 11:44:44.449166: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [37]:
model.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 1ms/step - loss: 13102691.0000 - accuracy: 0.1285


[13102691.0, 0.1285034418106079]

In [38]:
model.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 1ms/step - loss: 9641651.0000 - accuracy: 0.1163


[9641651.0, 0.11627907305955887]

In [39]:
#keras_test = ["a", "b", "c", "a", "c"]

In [40]:
#keras_train = tf.keras.utils.to_categorical(keras_test, num_classes=3)

In [41]:
set_seeds(101)
model2 = create_model(hl = 2, hu = 200, dropout = True, input_dim = len(columns_use), regularize = True)
model2.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 200, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [42]:
model2.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 1ms/step - loss: 55474012.0000 - accuracy: 0.1772


[55474012.0, 0.17715494334697723]

In [43]:
model2.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 1ms/step - loss: 65551060.0000 - accuracy: 0.1670


[65551060.0, 0.16701902449131012]

In [44]:
set_seeds(121)
model3 = create_model(hl = 2, hu = 600, dropout = True, input_dim = len(columns_use), regularize = True)
model3.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 200, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [45]:
model3.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 2ms/step - loss: 34546748.0000 - accuracy: 0.2533


[34546748.0, 0.2533051371574402]

In [46]:
model3.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 2ms/step - loss: 36411484.0000 - accuracy: 0.2135


[36411484.0, 0.21353065967559814]

In [47]:
set_seeds(121)
model4 = create_model(hl = 4, hu = 400, dropout = True, input_dim = len(columns_use), regularize = True)
model4.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 200, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [48]:
model4.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 2ms/step - loss: 31533956.0000 - accuracy: 0.2464


[31533956.0, 0.24643045663833618]

In [49]:
model4.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 2ms/step - loss: 22773680.0000 - accuracy: 0.2304


[22773680.0, 0.23044396936893463]

In [50]:
set_seeds(121)
model5 = create_model(hl = 4, hu = 700, dropout = True, input_dim = len(columns_use), regularize = True)
model5.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 200, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [51]:
model5.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 3ms/step - loss: 19090228.0000 - accuracy: 0.1824


[19090228.0, 0.18244315683841705]

In [52]:
model5.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 3ms/step - loss: 13944107.0000 - accuracy: 0.1734


[13944107.0, 0.17336152493953705]

In [53]:
set_seeds(121)
model6 = create_model(hl = 3, hu = 700, dropout = True, input_dim = len(columns_use), regularize = True)
model6.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 10, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [54]:
model6.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 2ms/step - loss: 148470224.0000 - accuracy: 0.1803


[148470224.0, 0.1803278625011444]

In [55]:
model6.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 2ms/step - loss: 123334152.0000 - accuracy: 0.1586


[123334152.0, 0.15856236219406128]

In [56]:
set_seeds(121)
model7 = create_model(hl = 4, hu = 1400, dropout = True, input_dim = len(columns_use), regularize = True)
model7.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 200, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [57]:
model7.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 7ms/step - loss: 6023089.0000 - accuracy: 0.2438


[6023089.0, 0.24378634989261627]

In [58]:
model7.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 7ms/step - loss: 4488968.5000 - accuracy: 0.1903


[4488968.5, 0.19027483463287354]

In [59]:
set_seeds(121)
model7a = create_model(hl = 4, hu = 1400, dropout = True, input_dim = len(columns_use), regularize = True)
model7a.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 100, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [60]:
model7a.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 7ms/step - loss: 34995776.0000 - accuracy: 0.2189


[34995776.0, 0.21893177926540375]

In [61]:
model7a.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 6ms/step - loss: 26066424.0000 - accuracy: 0.1903


[26066424.0, 0.19027483463287354]

In [62]:
set_seeds(121)
model7b = create_model(hl = 4, hu = 1400, dropout = True, input_dim = len(columns_use), regularize = True)
model7b.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 100, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [63]:
model7b.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 7ms/step - loss: 34995776.0000 - accuracy: 0.2189


[34995776.0, 0.21893177926540375]

In [64]:
model7b.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 6ms/step - loss: 26066424.0000 - accuracy: 0.1903


[26066424.0, 0.19027483463287354]

In [65]:
set_seeds(121)
model8 = create_model(hl = 2, hu = 1400, dropout = True, input_dim = len(columns_use), regularize = True)
model8.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 200, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [66]:
model8.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 4ms/step - loss: 16738583.0000 - accuracy: 0.2485


[16738583.0, 0.24854573607444763]

In [67]:
model8.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 4ms/step - loss: 14123707.0000 - accuracy: 0.1797


[14123707.0, 0.17970401048660278]

In [68]:
set_seeds(121)
model8a = create_model(hl = 2, hu = 1400, dropout = True, input_dim = len(columns_use), regularize = True)
model8a.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 300, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [69]:
model8a.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 4ms/step - loss: 4237091.5000 - accuracy: 0.3802


[4237091.5, 0.3802221119403839]

In [70]:
model8a.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 4ms/step - loss: 3281040.0000 - accuracy: 0.2727


[3281040.0, 0.27272728085517883]

In [71]:
set_seeds(121)
model9 = create_model(hl = 2, hu = 2100, dropout = True, input_dim = len(columns_use), regularize = True)
model9.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 200, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [72]:
model9.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 7ms/step - loss: 6238343.0000 - accuracy: 0.3310


[6238343.0, 0.33104178309440613]

In [73]:
model9.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 8ms/step - loss: 5147653.0000 - accuracy: 0.2008


[5147653.0, 0.2008456587791443]

In [74]:
set_seeds(121)
model10 = create_model(hl = 2, hu = 3000, dropout = True, input_dim = len(columns_use), regularize = True)
model10.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 100, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

KeyboardInterrupt: 

In [ ]:
model10.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

In [ ]:
model10.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

In [ ]:
set_seeds(121)
model11 = create_model(hl = 2, hu = 1400, dropout = True, input_dim = len(columns_use), regularize = True)
model11.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 400, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [ ]:
model11.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

In [ ]:
model11.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

In [ ]:
predictions = model8.predict(train_stand[columns_use])
predictions

In [ ]:
np.bincount(finan_stcks["PredictionGroup"])

In [ ]:
sum(np.bincount(finan_stcks["PredictionGroup"]))

In [ ]:
np.bincount(finan_stcks["PredictionGroup"][:split])


In [ ]:
grouper = list(np.bincount(finan_stcks["PredictionGroup"]))
print(grouper)
count = 0
for each in grouper:
    #each= float(each)
    grouper[count] = each/sum(np.bincount(finan_stcks["PredictionGroup"]))
    count +=1
print(grouper)

In [ ]:
grouper = list(np.bincount(finan_stcks["PredictionGroup"][:split]))
print(grouper)
count = 0
for each in grouper:
    #each= float(each)
    grouper[count] = each/sum(np.bincount(finan_stcks["PredictionGroup"][:split]))
    count +=1
print(grouper)

In [ ]:
grouper = list(np.bincount(finan_stcks["PredictionGroup"][split:]))
print(grouper)
count = 0
for each in grouper:
    #each= float(each)
    grouper[count] = each/sum(np.bincount(finan_stcks["PredictionGroup"][split:]))
    count +=1
print(grouper)

In [ ]:
list(np.bincount(finan_stcks["PredictionGroup"][split:]))


In [ ]:
[print(i.shape, i.dtype) for i in model.inputs]

In [ ]:
[print(o.shape, o.dtype) for o in model.outputs]

In [ ]:
[print(l.name, l.input_shape, l.dtype) for l in model.layers]